In [278]:
import math
import warnings
from datetime import datetime
start=datetime.now()
from IPython.display import display
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn import linear_model
import statsmodels.formula.api as smf
from sklearn.model_selection import cross_val_score
from sklearn import neighbors
from sklearn import tree,model_selection
from sklearn.grid_search import GridSearchCV
# A convenience for displaying visualizations.
from IPython.display import Image
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn import preprocessing
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif
from sklearn.ensemble import RandomForestRegressor
# Display preferences.
%matplotlib inline
pd.options.display.float_format = '{:.3f}'.format


In [279]:
# Suppress annoying harmless error.
warnings.filterwarnings(
    action="ignore",
    module="scipy",
    message="^internal gelsd"
)

from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
start=datetime.now()
rfc = ensemble.RandomForestRegressor()


We are creating dataframe of the the data in .csv format and are renaming the column name

In [280]:

data = pd.read_csv('C:/Personal/09142640/Downloads/Car_sales_U.csv')
data.columns.values[3]='Year_Resale_Value'
data_req=data.loc[:,["Sales_in_thousands","Year_Resale_Value","Price_in_thousands","Engine_size","Horsepower","Wheelbase","Width","Length","Curb_weight","Fuel_capacity","Fuel_efficiency","Power_perf_factor"]]
data_na=data_req.fillna(0)


Now, we are taking X and Y variable with X as input variable with the columns except 1 column which is for the output variable

In [281]:
Y = data_na[['Sales_in_thousands']].astype('int')
X = data_na.drop(columns='Sales_in_thousands').astype('int')
X.head(5)


,Year_Resale_Value,Price_in_thousands,Engine_size,Horsepower,Wheelbase,Width,Length,Curb_weight,Fuel_capacity,Fuel_efficiency,Power_perf_factor
0,16,21,1,140,101,67,172,2,13,28,58
1,19,28,3,225,108,70,192,3,17,25,91
2,18,0,3,225,106,70,192,3,17,26,0
3,29,42,3,210,114,71,196,3,18,22,91
4,22,23,1,150,102,68,178,2,16,27,62


Now, we will use Grid searchcv ,Holdout,Cross Validation to validate the accuracy of random forest model

In [282]:
# gridsearchCV

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
rfc = ensemble.RandomForestRegressor(max_depth=10,n_estimators= 100)
dt_grid={'max_depth':[3,4,5],'max_features': [11]}

grid_class=model_selection.GridSearchCV(rfc,dt_grid,cv=6)
grid_class.fit(X_train, y_train)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

GridSearchCV(cv=6, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 4, 5], 'max_features': [11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [283]:
#Displaying the parameters using Grid searchcv
results = grid_class.cv_results_
print(results.get('params'))
print(results.get('mean_test_score'))
print(results.get('mean_train_score'))
print(grid_class.best_params_)
print(grid_class.best_score_)
final_model = grid_class.best_estimator_
print(final_model)


[{'max_depth': 3, 'max_features': 11}, {'max_depth': 4, 'max_features': 11}, {'max_depth': 5, 'max_features': 11}]
[ 0.00073066 -0.0526419  -0.11281314]
[0.56321628 0.6437275  0.71150322]
{'max_depth': 3, 'max_features': 11}
0.0007306645747829444
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features=11, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [284]:
rfc.fit(X_train, y_train)
print('Training score with 20% Holdout: ' + str(rfc.score(X_train, y_train)))
print('Testing score with 20% Holdout: ' + str(rfc.score(X_test, y_test)))

Training score with 20% Holdout: 0.8409244427642835
Testing score with 20% Holdout: 0.05626668571158744


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


After using the holdout  groups, it looks like the training data is having good accuracy but the testing data is overfitting the model .We played around with max_depth=10,n_estimators= 100 and tried to change the value to get good accuracy and reduced overfitting values.Now, we will check the accuracy of the model with cross validation with folds=10.It should be wither 5 or 10, thats one of the better practices

In [285]:
#Cross Validation
from sklearn.model_selection import cross_val_score
print ("Training Data accuracy: ",cross_val_score(rfc, X_train,y_train, cv=5))
print ("Testing Data accuracy: ",cross_val_score(rfc, X_test,y_test, cv=5))

C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

Training Data accuracy:  [-0.11434606 -0.02640777 -0.0674444  -0.41735     0.12714916]


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


Testing Data accuracy:  [ 0.32436271 -0.17912002 -1.45733204  0.44325201 -4.67769448]


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


In [ ]:
After doing the cross validation,we are seeing overfitting in the dataset.

In [286]:
#Displaying the important feature scores

#decision_tree.fit(X_train, y_train)
rfc.fit(X_train, y_train)
print(rfc.feature_importances_)


[0.12961093 0.09507544 0.04504079 0.07148406 0.19286296 0.07604972
 0.21695685 0.01246968 0.05176627 0.04328333 0.06539996]


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


Now, we will try to get the best number of features, here we have observed that maximum percentage is with 8 features so we will take these 8 features

In [287]:
# And let's see what percentage of decisions are accounted for with the top X features.
importances = rfc.feature_importances_
indices = np.argsort(importances)[::-1]
top_indices = indices[:55]
for n_idx in range(2, 14, 2):
    print('Percentage of decisions accounted for with {} features: {}'.format(n_idx, importances[indices[:n_idx]].sum()))
print('Total Features: ' + str(len(indices)))

Percentage of decisions accounted for with 2 features: 0.40981980923079553
Percentage of decisions accounted for with 4 features: 0.6345061822317419
Percentage of decisions accounted for with 6 features: 0.7820399588227033
Percentage of decisions accounted for with 8 features: 0.8992061959299487
Percentage of decisions accounted for with 10 features: 0.9875303158974308
Percentage of decisions accounted for with 12 features: 1.0
Total Features: 11


Let's reduce to 8 features since we could potentially account for 93% of the decisions from the Random Forest.

In [288]:
#Using PCA
n_indices = 8
X_train.columns[indices[:n_indices]]


Index(['Length', 'Wheelbase', 'Year_Resale_Value', 'Price_in_thousands',
       'Width', 'Horsepower', 'Power_perf_factor', 'Fuel_capacity'],
      dtype='object')

In [289]:
X_train_rfc = X_train.loc[:, X_train.columns[indices[:n_indices]]]
X_test_rfc = X_test.loc[:, X_train.columns[indices[:n_indices]]]
print(X_train_rfc.shape, X_test_rfc.shape)

(124, 8) (32, 8)


In [290]:
X_pca = PCA(n_components=8)
X_pca.fit(X_train)
X_train_pca = X_pca.transform(X_train)
X_test_pca = X_pca.transform(X_test)


In [291]:
print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    X_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.88727673 0.06529464 0.02277412 0.01125091 0.00568839 0.00338902
 0.00192852 0.00131014]


By using PCA, we can see 8 features are closely correlated to the output variable so we will keep them
Now we will use these models after doing PCA-

1.GridsearchCV

2.R-squared

3.Mean Square Error


In [292]:
#Using Grid search  for finding the best set of parameters with random Forest model
dt_grid_pca={'max_depth':[10],'max_features': [8]}
grid_DT_pca=model_selection.GridSearchCV(rfc,dt_grid_pca,cv=6)
grid_DT_pca.fit(X_train, y_train)

#grid_class_pca=model_selection.GridSearchCV(random_f_pca,dt_grid_pca,refit=True,return_train_score=True)
#grid_class_pca.fit(X_train_pca, y_train_pca)
#Displaying the parameters using Grid search
results_pca = grid_DT_pca.cv_results_
print(results_pca.get('params'))
print(results_pca.get('mean_test_score'))
print(results_pca.get('mean_train_score'))
print(grid_DT_pca.best_params_)
print(grid_DT_pca.best_score_)
final_model_pca = grid_DT_pca.best_estimator_
print(final_model_pca)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

[{'max_depth': 10, 'max_features': 8}]
[-0.19501099]
[0.86140985]
{'max_depth': 10, 'max_features': 8}
-0.19501099291853002
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [293]:
# Calculating model accuracy using R square
print("R-squared :" ,grid_DT_pca.score(X_train,y_train))


R-squared : 0.8583785303053476


It looks like the R-squared got 85.8% accuracy.Now we will determine mean square error

In [294]:
#Validating model accuracy using mean square error

y_pred=grid_DT_pca.predict(X_train)
print("Mean Squared Error: ",metrics.mean_squared_error(y_train,y_pred))

Mean Squared Error:  699.2672803347574


It looks like the mean squared error is medium,not so hight not so low.

We have completed feature selection using PCA where we used grid searchcv , R-squared and mean squared error to check the accuracy.Now we are going to use selectkbest to get the optimum features for  the feature selection and will use the same methods to validate the accuracy of the model

In [295]:
n_indices = 8
X_train.columns[indices[:n_indices]]

Index(['Length', 'Wheelbase', 'Year_Resale_Value', 'Price_in_thousands',
       'Width', 'Horsepower', 'Power_perf_factor', 'Fuel_capacity'],
      dtype='object')

In [296]:
#selectkbest to get the scores of the best parameters
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)
names = X_train.columns[indices[:n_indices]]
skb_test = SelectKBest(score_func=f_regression,k=n_indices)
X_train_kbest = skb_test.fit_transform(X_train, y_train)
for n in range(0, len(names)):
 print(names[n], skb_test.scores_[n])


Length 8.438299882694018
Wheelbase 9.442760665349974
Year_Resale_Value 0.032121195318380376
Price_in_thousands 3.8193782923248536
Width 20.339651897458115
Horsepower 3.531395140114711
Power_perf_factor 10.998690940404886
Fuel_capacity 0.001409436087695959


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Using selectkbest , we have got 8 best features. Now we will use these methods  to validate the accuracy of the model


1.GridsearchCV

2.R-squared

3.Mean Square Error



In [297]:
#Gridsearchcv
n_indices = 8
X_train.columns[indices[:n_indices]]
skb = SelectKBest(k=n_indices)
skb.fit(X_train, y_train)
X_train_skb = skb.transform(X_train)
X_test_skb = skb.transform(X_test)

C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Usually its a good practice to assign max depth as highest as possible.Here we have chose it as 50.Max_features is 11.Now,we will gridsearchcv to get the best set of parameters 

In [298]:
#Using Grid search  for finding the best set of parameters with random Forest model

dt_grid_skb={'max_depth':[10],'max_features': [11]}
grid_DT_skb=model_selection.GridSearchCV(rfc,dt_grid,cv=6)
grid_DT_skb.fit(X_train, y_train)

#grid_class_pca=model_selection.GridSearchCV(random_f_pca,dt_grid_pca,refit=True,return_train_score=True)
#grid_class_pca.fit(X_train_pca, y_train_pca)
#Displaying the parameters using Grid search
results_skb = grid_DT_pca.cv_results_
print(results_skb.get('params'))
print(results_skb.get('mean_test_score'))
print(results_skb.get('mean_train_score'))
print(grid_DT_skb.best_params_)
print(grid_DT_skb.best_score_)
final_model_pca = grid_DT_pca.best_estimator_
print(final_model_pca)

C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

[{'max_depth': 10, 'max_features': 8}]
[-0.19501099]
[0.86140985]
{'max_depth': 3, 'max_features': 11}
0.0012226642847281794
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=8, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:740: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  self.best_estimator_.fit(X, y, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [300]:
# Calculate R-squared to get the accuracy
print("R-squared :" ,grid_DT_skb.score(X_train,y_train))



R-squared : 0.5451488180352453


The R-squared  accuracy is little low.so looks like PCA is better to get the accuracy of the model

In [302]:
#Validating model accuracy using mean square error

y_pred_skb=grid_DT_skb.predict(X_train)
print("Mean Squared Error: ",metrics.mean_squared_error(y_train,y_pred_skb))

Mean Squared Error:  2396.367220549899


In [ ]:
The mean squared error here for Seleckbest feature selection came to be on the higher side.

In [303]:
#Total time took to run the entire model
print(datetime.now()-start)


0:03:46.306687


The accuracy for this model is 86% which is good.But there is overfitting in the variables.We played with the paramters to decrease the overfitting ,it gets reduced a little but never goes away.We used feature selections such as PCA/Selectkbest which provided us more information about the paramters but didn't do that well in accuracy of the mode.Total time to run this model took 4 minutes,so it is not that slow, lets compare it with the Decision tree

# Decision Tree

Now, we will use Grid searchcv ,Holdout,Cross Validation to validate the accuracy of Decision Tree model

In [304]:
# gridsearchCV to get  the best parameters of the model
decision_tree = tree.DecisionTreeRegressor(max_depth=50)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)
dt_grid={'max_depth':[50],'max_features': [11]}
grid_class_dt=model_selection.GridSearchCV(rfc,dt_grid,cv=6)
grid_class_dt.fit(X_train, y_train)



C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:458: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
C:\Users\09142640\AppData\Local\Continuum\anacond

GridSearchCV(cv=6, error_score='raise',
       estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [50], 'max_features': [11]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [305]:
#Displaying the parameters using Grid searchcv
results = grid_class_dt.cv_results_
print(results.get('params'))
print(results.get('mean_test_score'))
print(results.get('mean_train_score'))
print(grid_class.best_params_)
print(grid_class.best_score_)
final_model = grid_class.best_estimator_
print(final_model)

[{'max_depth': 50, 'max_features': 11}]
[-0.09585207]
[0.85998853]
{'max_depth': 3, 'max_features': 11}
0.0007306645747829444
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=3,
           max_features=11, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=100, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [306]:
#Holdout Groups to get the accuracy of the model
decision_tree.fit(X_train, y_train)
print('Training score with 20% Holdout: ' + str(decision_tree.score(X_train, y_train)))
print('Testing score with 20% Holdout: ' + str(decision_tree.score(X_test, y_test)))

Training score with 20% Holdout: 0.9983320100163819
Testing score with 20% Holdout: -1.8280346912491514


Looks like decision tree accuracy is very high which is 99.9% for the training data.But it looks like we have overfitting for the testing data.If there is any overfitting there is no option to remove it.

In [307]:
#Cross Validation
from sklearn.model_selection import cross_val_score
print ("Training Data accuracy: ",cross_val_score(decision_tree, X_train,y_train, cv=5))
print ("Testing Data accuracy: ",cross_val_score(decision_tree, X_test,y_test, cv=5))

Training Data accuracy:  [-3.27662965 -0.08601479 -1.08286987 -0.50237446 -0.44275607]
Testing Data accuracy:  [-0.57312993 -0.35709988 -4.32987185  0.71389126 -0.48725284]


After doing the cross validation,we are seeing overfitting in the dataset.

In [308]:
#Displaying the important feature scores
print(decision_tree.feature_importances_)


[0.02434791 0.15421578 0.01313339 0.01983086 0.37803859 0.06393907
 0.09461163 0.01422038 0.07455922 0.10859569 0.05450748]


Now, we will try to get the best number of features as mentioned below

In [309]:
# And let's see what percentage of decisions are accounted for with the top X features.
importances = decision_tree.feature_importances_
indices = np.argsort(importances)[::-1]
top_indices = indices[:55]
for n_idx in range(2, 14, 2):
    print('Percentage of decisions accounted for with {} features: {}'.format(n_idx, importances[indices[:n_idx]].sum()))
print('Total Features: ' + str(len(indices)))

Percentage of decisions accounted for with 2 features: 0.5322543663161408
Percentage of decisions accounted for with 4 features: 0.7354616840385361
Percentage of decisions accounted for with 6 features: 0.8739599764462221
Percentage of decisions accounted for with 8 features: 0.9528153654094783
Percentage of decisions accounted for with 10 features: 0.9868666063376076
Percentage of decisions accounted for with 12 features: 1.0000000000000002
Total Features: 11


As noticed above. we have observed that  out of all the features 8 features are the most important ones with  97% of the Decision tree,so we will ignore the rest.

First we will use PCA for the 8 features to 

In [310]:
#Using PCA
n_indices = 8
X_train.columns[indices[:n_indices]]

Index(['Wheelbase', 'Price_in_thousands', 'Fuel_efficiency', 'Length',
       'Fuel_capacity', 'Width', 'Power_perf_factor', 'Year_Resale_Value'],
      dtype='object')

In [311]:
X_pca = PCA(n_components=8)
X_pca.fit(X_train)
X_train_pca = X_pca.transform(X_train)
X_test_pca = X_pca.transform(X_test)


In [312]:
print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    X_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.89263658 0.06091915 0.02342764 0.01054047 0.00516519 0.00300382
 0.00236456 0.00100843]


The above shows that variance explained by 8 variables in the model.It looks like the first variable explains the most in the model and followed by rest of the variables

By using PCA, we can see 8 features are closely correlated to the output variable so we will keep them Now we will use these models after doing PCA-

1.GridsearchCV

2.R-squared

3.Mean Square Error


In [313]:
#Using Grid search  for finding the best set of parameters with random Forest model
dt_grid_pca={'max_depth':[10],'max_features': [8]}
grid_dt_pca=model_selection.GridSearchCV(decision_tree,dt_grid_pca,cv=6)
grid_dt_pca.fit(X_train, y_train)

#grid_class_pca=model_selection.GridSearchCV(random_f_pca,dt_grid_pca,refit=True,return_train_score=True)
#grid_class_pca.fit(X_train_pca, y_train_pca)
#Displaying the parameters using Grid search
results_pca_dt = grid_dt_pca.cv_results_
print(results_pca.get('params'))
print(results_pca.get('mean_test_score'))
print(results_pca.get('mean_train_score'))
print(grid_dt_pca.best_params_)
print(grid_dt_pca.best_score_)
final_model_pca = grid_dt_pca.best_estimator_
print(final_model_pca)

[{'max_depth': 10, 'max_features': 8}]
[-0.19501099]
[0.86140985]
{'max_depth': 10, 'max_features': 8}
-0.9096220106387761
DecisionTreeRegressor(criterion='mse', max_depth=10, max_features=8,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [316]:
# Calculating model accuracy using R square
print("R-squared :" ,grid_dt_pca.score(X_train,y_train))


R-squared : 0.8791756547134039


It looks like the R-squared got 87.9% accuracy which is good.Now we will determine mean square error

In [317]:
#Validating model accuracy using mean square error

y_pred=grid_dt_pca.predict(X_train)
print("Mean Squared Error: ",metrics.mean_squared_error(y_train,y_pred))

Mean Squared Error:  662.7409824046921


It looks like the mean squared error is very low.

We have completed feature selection using PCA where we used grid searchcv , R-squared and mean squared error to check the accuracy.Now we are going to use selectkbest to get the optimum features for the feature selection and will use the same methods to validate the accuracy of the model

In [267]:
n_indices = 8
X_train.columns[indices[:n_indices]]

Index(['Length', 'Year_Resale_Value', 'Horsepower', 'Price_in_thousands',
       'Width', 'Wheelbase', 'Power_perf_factor', 'Engine_size'],
      dtype='object')

In [318]:
#selectkbest to get the scores of the best parameters
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_regression
#X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=1)
names = X_train.columns[indices[:n_indices]]
skb_test_dt = SelectKBest(score_func=f_regression,k=n_indices)
X_train_kbest = skb_test_dt.fit_transform(X_train, y_train)
for n in range(0, len(names)):
 print(names[n], skb_test_dt.scores_[n])


Wheelbase 9.811881430592454
Price_in_thousands 9.704322194063911
Fuel_efficiency 0.2236537612958624
Length 3.7185452745498724
Fuel_capacity 21.18935199538139
Width 4.608615055041465
Power_perf_factor 10.534657365839875
Year_Resale_Value 0.25822026672355836


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Using selectkbest , we have got 8 best features. Now we will use these methods to validate the accuracy of the model

1.GridsearchCV

2.R-squared

3.Mean Square Error


In [ ]:
#Gridsearchcv
n_indices = 8
X_train.columns[indices[:n_indices]]
skb_dt = SelectKBest(k=n_indices)
skb_dt.fit(X_train, y_train)
X_train_skb = skb_dt.transform(X_train)
X_test_skb = skb_dt.transform(X_test)

Usually its a good practice to assign max depth as highest as possible.Here we have chose it as 50.Max_features is 11.Now,we will gridsearchcv to get the best set of parameters 

In [272]:
dt_grid_skb={'max_depth':[50],'max_features': [11]}
grid_dt_skb=model_selection.GridSearchCV(decision_tree,dt_grid,cv=6)
grid_dt_skb.fit(X_train, y_train)

#grid_class_pca=model_selection.GridSearchCV(random_f_pca,dt_grid_pca,refit=True,return_train_score=True)
#grid_class_pca.fit(X_train_pca, y_train_pca)
#Displaying the parameters using Grid search
results_skb_dt = grid_dt_skb.cv_results_
print(results_skb.get('params'))
print(results_skb.get('mean_test_score'))
print(results_skb.get('mean_train_score'))
print(grid_dt_skb.best_params_)
print(grid_dt_skb.best_score_)
final_model_skb = grid_dt_skb.best_estimator_
print(final_model_skb)

[{'max_depth': 10, 'max_features': 8}]
[-0.03784745]
[0.85985436]
{'max_depth': 50, 'max_features': 11}
-0.8352723644127859
DecisionTreeRegressor(criterion='mse', max_depth=50, max_features=11,
           max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           presort=False, random_state=None, splitter='best')


C:\Users\09142640\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:122: FutureWarning: You are accessing a training score ('mean_train_score'), which will not be available by default any more in 0.21. If you need training scores, please set return_train_score=True
  warnings.warn(*warn_args, **warn_kwargs)


In [319]:
# Calculate R-squared to get the accuracy
print("R-squared :" ,grid_dt_skb.score(X_train,y_train))



R-squared : 0.8744979048156781


It looks like R-squared is very accurate with accuracy of 87.44%.Now lets go for mean squared error calculation

In [320]:
y_pred_skb=grid_dt_skb.predict(X_train)
print("Mean Squared Error: ",metrics.mean_squared_error(y_train,y_pred_skb))

Mean Squared Error:  688.3991935483871


As seen above,mean squared error is very low as compared to PCA, which means the accuracy is very high if coompared to PCA

In [321]:
#Total time took to run the entire model
print(datetime.now()-start)

0:08:56.415570


The accuracy for this model is 87.4% which is good.Selectkbest provides good accuracy for Decision Tree.But there is overfitting in the variables.We played with the paramters to decrease the overfitting ,it gets reduced a little but never goes away.We used feature selections such as PCA/Selectkbest which provided us more information about the paramters .Total time to run this model took 4 minutes,so it is not that slow

# Conclusion:

The model Decision Tree is able to perform  better than Random Forest. The dataset (test and training ) has some overfitting which can be reduced but not eliminated completely. Decision tree was the top scorer, with 0.87 on training data  with R-squrared.Random Forest has the score of 0.858 on training data.Decision tree took more time than Random Forest to run the model where Decision tree took 8 minutes and Random Forest took 3 minutes respectively.Using PCA and SelectKbest, the feature selection was performed and featured were reduced to 8.Since, the score was on the higher side, so we assume that all the 8 features were able to explicitly explain most of the variance in the model.